## `builtin`

除`zipline`标准因子、过滤器、分类器外，增加`builtin`模块，将基础数据与标准`pipeline`整合，修改或模拟`quantopian`IDE功能。主要包括以下部分：
+ 自定义因子
+ 自定义过滤器
+ 自定义分类器

In [1]:
from zipline.pipeline.fundamentals.reader import Fundamentals
from zipline.pipeline.builtin import TradingDays,QTradableStocks
from zipline.research import run_pipeline, select_output_by
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

### 因子

In [2]:
from zipline.pipeline.builtin import TradingDays

#### `TradingDays` 期间交易天数
+ 当天成交量大约0,有效交易
+ 需要指定`window_length`

In [3]:
def make_pipeline():
    t20 = TradingDays(window_length=20)
    t200 = TradingDays(window_length=200)
    return Pipeline(
        columns={
            '20天内有效交易天数': t20,
            '200天内有效交易天数': t200,           
        }, 
    )

In [4]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [5]:
select_output_by(result,'2018-04-23','2018-04-24',stock_codes=['000001','600645','600076'])

200天内有效交易天数  20天内有效交易天数
2018-04-23 00:00:00+00:00 平安银行(000001)        200.0        20.0
                          康欣新材(600076)        151.0         0.0
                          中源协和(600645)        127.0        20.0
2018-04-24 00:00:00+00:00 平安银行(000001)        200.0        20.0
                          康欣新材(600076)        150.0         0.0
                          中源协和(600645)        127.0        20.0

+ 600076期间停牌三个月，20日内无交易，但200天内存在交易
+ 600645在200天内有停牌，有效成交率不足90%
+ 000001每天正常交易

### 过滤器

#### `IsST`
+ 当前是否为ST状态

In [6]:
from zipline.pipeline.builtin import IsST

In [7]:
def make_pipeline():
    is_st = IsST()
    # 用于mask参数，非st计算结果为NaN
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], mask=is_st, window_length=200)
    return Pipeline(
        columns={
            '平均收盘': ma20,
        }, )

In [8]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [9]:
select_output_by(
    result, '2018-04-23', stock_codes=['600408', '600645', '600076'])

平均收盘
2018-04-23 00:00:00+00:00 康欣新材(600076)       NaN
                          *ST安泰(600408)      NaN
                          中源协和(600645)       NaN
2018-04-24 00:00:00+00:00 康欣新材(600076)       NaN
                          *ST安泰(600408)  3.41749
                          中源协和(600645)       NaN
2018-04-25 00:00:00+00:00 康欣新材(600076)       NaN
                          *ST安泰(600408)  3.40904
                          中源协和(600645)       NaN

In [10]:
def make_pipeline():
    is_st = IsST()
    ma20 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=200)
    # 用于screen参数，非st不会显示
    return Pipeline(
        columns={
            '平均收盘': ma20,
        }, 
        screen=is_st
    )

In [11]:
result = run_pipeline(make_pipeline(), '2018-4-20', '2018-4-26')

In [12]:
select_output_by(
    result, '2018-04-23', stock_codes=['600408', '600645', '600076'])

,,平均收盘
2018-04-24 00:00:00+00:00,*ST安泰(600408),3.41749
2018-04-25 00:00:00+00:00,*ST安泰(600408),3.40904
